In [2]:
!pip install -q transformers datasets scikit-learn pandas

In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score
from transformers import pipeline

In [10]:
data = [
    # positive
    ("I love this movie, it was amazing!", "positive"),
    ("This song makes me so happy.", "positive"),
    ("The food was really good and I enjoyed it.", "positive"),
    ("I had a great day today.", "positive"),
    ("The lecture was clear and helpful.", "positive"),
    ("I feel satisfied with the result.", "positive"),
    ("The new update is actually pretty nice.", "positive"),
    ("My friend was so kind to me today.", "positive"),
    ("The weather is beautiful and I like it.", "positive"),
    ("This app is very useful and easy to use.", "positive"),
    ("The cafe was cozy and the staff were friendly.", "positive"),
    ("I didn’t expect much, but it turned out great.", "positive"),
    ("The presentation went better than I thought.", "positive"),
    ("I’m proud of what I did today.", "positive"),
    ("The service was slow, but overall I’m happy.", "positive"),
    ("It wasn’t perfect, but I still enjoyed it.", "positive"),
    ("The game was fun and exciting.", "positive"),
    ("I feel calm and content right now.", "positive"),
    ("The new feature is surprisingly helpful.", "positive"),
    ("Even though I was tired, the day felt rewarding.", "positive"),

    # negative
    ("I hate this movie, it was terrible.", "negative"),
    ("This is the worst experience ever.", "negative"),
    ("The food tasted bad and disgusting.", "negative"),
    ("I am really disappointed with the service.", "negative"),
    ("Today was such a horrible day.", "negative"),
    ("The lecture was boring and confusing.", "negative"),
    ("I feel so upset about the result.", "negative"),
    ("The new update is really annoying.", "negative"),
    ("My friend ignored me and I feel bad.", "negative"),
    ("The weather is awful and I hate it.", "negative"),
    ("I thought it would be good, but it was actually pretty bad.", "negative"),
    ("The app keeps crashing and it’s frustrating.", "negative"),
    ("It wasn’t the worst, but I can’t say I liked it.", "negative"),
    ("The more I use this, the more disappointed I feel.", "negative"),
    ("I’m tired and nothing went well today.", "negative"),
    ("The staff were rude and unhelpful.", "negative"),
    ("I tried to enjoy it, but I just couldn’t.", "negative"),
    ("The result is far from what I expected.", "negative"),
    ("I feel anxious and unhappy about this.", "negative"),
    ("Even though some parts were okay, overall it was bad.", "negative"),
]

df = pd.DataFrame(data, columns=["text", "label"])
len(df)


40

In [11]:
def rule_based_sentiment(text: str) -> str:
    t = text.lower()

    positive_keywords = ["love", "happy", "amazing", "great", "good", "nice", "satisfied", "beautiful", "useful", "kind", "enjoyed", "helpful"]
    negative_keywords = ["hate", "worst", "terrible", "bad", "disappointed", "horrible", "boring", "upset", "annoying", "awful", "disgusting", "ignored"]

    # Count how many positive / negative words appear
    pos_score = sum(kw in t for kw in positive_keywords)
    neg_score = sum(kw in t for kw in negative_keywords)

    if pos_score > neg_score:
        return "positive"
    elif neg_score > pos_score:
        return "negative"
    else:
        # default guess when it's unclear
        return "negative"

# quick test
print(rule_based_sentiment("I love this movie but the ending was bad"))


negative


In [12]:
texts = df["text"].tolist()
true_labels = df["label"].tolist()

baseline_preds = [rule_based_sentiment(t) for t in texts]
baseline_acc = accuracy_score(true_labels, baseline_preds)

baseline_acc

0.8

In [13]:
sentiment_model = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

Device set to use cpu


In [14]:
# Run the model on all texts
hf_outputs = sentiment_model(df["text"].tolist())

# Show the first few results
hf_outputs[:5]


[{'label': 'POSITIVE', 'score': 0.9998791217803955},
 {'label': 'POSITIVE', 'score': 0.9998816251754761},
 {'label': 'POSITIVE', 'score': 0.9998832941055298},
 {'label': 'POSITIVE', 'score': 0.9998636245727539},
 {'label': 'POSITIVE', 'score': 0.9997597336769104}]

In [15]:
def hf_label_to_simple(label: str) -> str:
    return "positive" if label.upper() == "POSITIVE" else "negative"

hf_preds = [hf_label_to_simple(o["label"]) for o in hf_outputs]

hf_acc = accuracy_score(df["label"], hf_preds)
hf_acc


1.0

In [16]:
print("Baseline accuracy:", baseline_acc)
print("AI pipeline accuracy:", hf_acc)
print()

print("Examples where baseline and AI differ:\n")
for text, true, base, hf in zip(df["text"], df["label"], baseline_preds, hf_preds):
    if base != hf:
        print("TEXT:", text)
        print("TRUE LABEL:", true)
        print("BASELINE:", base)
        print("AI MODEL:", hf)
        print("-" * 60)


Baseline accuracy: 0.8
AI pipeline accuracy: 1.0

Examples where baseline and AI differ:

TEXT: The cafe was cozy and the staff were friendly.
TRUE LABEL: positive
BASELINE: negative
AI MODEL: positive
------------------------------------------------------------
TEXT: The presentation went better than I thought.
TRUE LABEL: positive
BASELINE: negative
AI MODEL: positive
------------------------------------------------------------
TEXT: I’m proud of what I did today.
TRUE LABEL: positive
BASELINE: negative
AI MODEL: positive
------------------------------------------------------------
TEXT: The game was fun and exciting.
TRUE LABEL: positive
BASELINE: negative
AI MODEL: positive
------------------------------------------------------------
TEXT: I feel calm and content right now.
TRUE LABEL: positive
BASELINE: negative
AI MODEL: positive
------------------------------------------------------------
TEXT: Even though I was tired, the day felt rewarding.
TRUE LABEL: positive
BASELINE: negat